# Imports

# Constants

# Visualizing Feature Space

## Visual Feature Analysis

### n-gram viewer

### Network visualization

### Co-occurrence plots

### Text x-rays and dispersion plots

## Guided Feature Engineering

### Part-of-speech tagging

### Most informative features

# Model Diagnostics

## Visualizing Clusters

## Visualizing Classes

## Diagnosing Classification Error

### Classification report heatmaps

### Confusion matrices

# Visual Steering

## Silhouette Scores and Elbow Curves

### Silhouette scores

### Elbow curves